In [44]:
import pandas as pandas
import numpy as numpy
import nltk
import re
import collections
import bisect
from nltk.tokenize import RegexpTokenizer

nltk.download('stopwords')
resultado = pandas.read_csv('https://raw.githubusercontent.com/AgnaldoCC/lab_6_recinfo/master/results.csv')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**1. Reconstruir o índice considerando o conjunto de dados que indicamos. Esses são os dados coletados por Bernardi e os estaremos usando para facilitar a correção da atividade. Se você já estiver usando esses dados não precisa reconstruir o índice;**



In [45]:
tokens = RegexpTokenizer(r'([A-Za-zÁáÉéÍíÓóÚúÃãÕõÇçÂâÊê]{3,27})')
stopwords = nltk.corpus.stopwords.words('portuguese') 
indices = {}
n = 0

for text in resultado.text:
  words = [word for word in tokens.tokenize(text.lower())
           if not bool(re.search(r'\d', word))
           and word not in stopwords and len(word) >= 3]  
  n += 1
  for t in words:
    if t not in indices.keys():
      indices[t] = []
    indices[t].append(n)
    
for elem in indices.items():
  d = dict(collections.Counter(elem[1]))
  indices[elem[0]] = list(d.items())

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:9: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  if __name__ == '__main__':


In [46]:
indices_df = pandas.DataFrame()
indices_df['Word'] = indices.keys()
indices_df['Documents and Frequencies'] = indices.values()

indices_df.head(10)

,Word,Documents and Frequencies
0,obrigaria,"[(91, 1), (213, 1)]"
1,meloso,"[(186, 1)]"
2,bloqueou,"[(69, 1), (206, 1)]"
3,explodindo,"[(11, 1)]"
4,espaços,"[(38, 1), (166, 1), (7, 1), (169, 1), (11, 1),..."
5,suzano,"[(214, 5), (239, 7)]"
6,francesco,"[(73, 1), (97, 1)]"
7,impotente,"[(29, 1)]"
8,prazo,"[(128, 1), (129, 2), (70, 1), (65, 2), (137, 2..."
9,paternalista,"[(11, 1)]"


**2. Refinar o índice invertido de forma a também incluir o IDF (inverse document frequency) de cada termo do dicionário;**

In [0]:
M = resultado.text.count()
for word in indices:
  k = len(indices[word])
  IDF = round(numpy.log((M+1)/k),2)
  indices[word].append(IDF)

In [48]:
indices_df['IDF'] = [index[-1] for index in indices.values()]

indices_df.head(10)

,Word,Documents and Frequencies,IDF
0,obrigaria,"[(91, 1), (213, 1), 4.83]",4.83
1,meloso,"[(186, 1), 5.52]",5.52
2,bloqueou,"[(69, 1), (206, 1), 4.83]",4.83
3,explodindo,"[(11, 1), 5.52]",5.52
4,espaços,"[(38, 1), (166, 1), (7, 1), (169, 1), (11, 1),...",2.77
5,suzano,"[(214, 5), (239, 7), 4.83]",4.83
6,francesco,"[(73, 1), (97, 1), 4.83]",4.83
7,impotente,"[(29, 1), 5.52]",5.52
8,prazo,"[(128, 1), (129, 2), (70, 1), (65, 2), (137, 2...",2.30
9,paternalista,"[(11, 1), 5.52]",5.52


**3. Implementar as seguintes versões do modelo vetorial:**

**3.1. Representação binária**

In [0]:
def binaria(query, document):
  score = 0
  query_tokens = query.split()
  doc_tokens = document.split()
  
  for token in query_tokens:
    score += (token in doc_tokens)
    
  return score

**3.2. TF (lembre-se que esse modelo já está implementado)**

In [0]:
def tf(query, document):
  score = 0
  doc_tokens = document.split()
  query_tokens = query.split()
  
  for word in query_tokens:
    score += doc_tokens.count(word)
  
  return score

**3.3 TF-IDF**

In [0]:
def tfidf(query, document):
  score = 0
  doc_tokens = document.split()
  query_tokens = query.split()
  
  for word in query_tokens:
    cwd = doc_tokens.count(word)
    if word in indices:
      score += cwd * indices[word][-1]
  
  return round(score,2)

**3.4 BM25 (não usaremos Okapi já que os documentos não tem grande variação de tamanho)**

In [0]:
def bm25(query, document, k):
  score = 0
  doc_tokens = document.split()
  query_tokens = query.split()
  
  words = [word for word in query_tokens if word in doc_tokens]
    
  for word in words:
    cwd = doc_tokens.count(word)
    dfw = 0
    if word in indices:
      dfw = len(indices[word][:-1])
    score += (((k+1) * cwd) / (cwd + k)) * numpy.log10(((M+1) / dfw)) if dfw != 0 else 0
  
  return round(score,2)

**4. Execute os algoritmos separadamente em 3 consultas de sua escolha e retorne os top-5 documentos mais similares à cada consulta;**

In [0]:
queries = ['bolsonaro', 'futebol', 'ministro']

In [0]:
def cria_modelos(query):
  n = 0
  db = []
  dtf = []
  dtfidf = []
  dbm25 = []
  for doc in resultado.text:
    doc = doc.lower()
    n += 1
    bisect.insort(db, (binaria(query, doc), n))
    bisect.insort(dtf, (tf(query,doc), n))
    bisect.insort(dtfidf, (tfidf(query,doc), n))
    bisect.insort(dbm25, (bm25(query,doc,20), n))
  
  db.reverse()
  dtf.reverse()
  dtfidf.reverse()
  dbm25.reverse()
  
  return db[:5], dtf[:5], dtfidf[:5], dbm25[:5]

In [0]:
top5_binaria = ['','','']
top5_tf = ['','','']
top5_tfidf = ['','','']
top5_bm25 = ['','','']

top5_binaria[0], top5_tf[0], top5_tfidf[0], top5_bm25[0] = cria_modelos(queries[0])
top5_binaria[1], top5_tf[1], top5_tfidf[1], top5_bm25[1] = cria_modelos(queries[1])
top5_binaria[2], top5_tf[2], top5_tfidf[2], top5_bm25[2] = cria_modelos(queries[2])

In [56]:
query_df = pandas.DataFrame()

query_df['Query'] = queries
query_df['Binaria'] = top5_binaria
query_df['TF'] = top5_tf
query_df['TF-IDF'] = top5_tfidf
query_df['BM25'] = top5_bm25

query_df.index+=1
query_df

,Query,Binaria,TF,TF-IDF,BM25
1,bolsonaro,"[(1, 248), (1, 240), (1, 238), (1, 237), (1, 2...","[(32, 151), (30, 207), (30, 166), (19, 19), (1...","[(35.2, 151), (33.0, 207), (33.0, 166), (20.9,...","[(5.73, 207), (5.73, 166), (5.73, 151), (4.77,..."
2,futebol,"[(1, 243), (1, 242), (1, 212), (1, 189), (1, 1...","[(7, 242), (7, 47), (6, 115), (5, 118), (3, 212)]","[(16.8, 242), (16.8, 47), (14.4, 115), (12.0, ...","[(5.21, 242), (5.21, 47), (4.17, 118), (4.17, ..."
3,ministro,"[(1, 247), (1, 246), (1, 240), (1, 236), (1, 2...","[(11, 222), (7, 240), (7, 209), (6, 215), (5, ...","[(15.29, 222), (9.73, 240), (9.73, 209), (8.34...","[(4.21, 222), (3.01, 240), (3.01, 209), (2.41,..."


**5. Compare os resultados encontrados e responda.**

**5.1. Quais modelos você acha que trouxe os melhores resultados? Por que? Inspecione os documentos retornados para melhor embasar sua resposta.**

Após a analise realizada sobre os resultados, os modelos TF-IDF e BM25 apresentaram os melhores resultados, sendo ambos bem semelhantes. Mas, se fosse para escolher um método, levando em conta todas as variáveis, o melhor seria o TF-IDF, já que apresenta um resultado satisfatório e é menos complexo que o BM25.

**5.2. Calcule e reporte o overlap par-a-par entre os resultados de cada modelo (usando o índice de Jaccard)**

In [0]:
def intersection_size(a,b):
  return len([elem for elem in a if elem in b])

def indice_jaccard(a,b):
  l_a = len(a)
  l_b = len(b)
  l_ab = intersection_size(a,b)
  exp = l_a + l_b - l_ab
  jaccard = l_ab / exp if exp != 0 else 0
  return jaccard

In [0]:
def get_top1_score(top5):
  return [top[0][0] for top in top5]

def get_top1_doc(top5):
  return [top[0][1] for top in top5]

def get_doc_title(docs):
  return [resultado.title[doc] for doc in docs]

In [0]:
score_top5_binaria = get_top1_score(top5_binaria)
score_top5_tf = get_top1_score(top5_tf)
score_top5_tfidf = get_top1_score(top5_tfidf)
score_top5_bm25 = get_top1_score(top5_bm25)

doc_top5_binaria = get_top1_doc(top5_binaria)
doc_top5_tf = get_top1_doc(top5_tf)
doc_top5_tfidf = get_top1_doc(top5_tfidf)
doc_top5_bm25 = get_top1_doc(top5_bm25)

titles_binaria = get_doc_title(doc_top5_binaria)
titles_tf = get_doc_title(doc_top5_tf)
titles_tfidf = get_doc_title(doc_top5_tfidf)
titles_bm25 = get_doc_title(doc_top5_bm25)

In [60]:
all_docs = []
all_docs.append(doc_top5_binaria)
all_docs.append(doc_top5_bm25)
all_docs.append(doc_top5_tf)
all_docs.append(doc_top5_tfidf)

matriz = [['Measure', 'Binary', 'TF', 'TF-IDF', 'BM25']]
titles = matriz
for i in range(len(all_docs)):
  line = []
  for j in range(len(all_docs)):
    jaccard = indice_jaccard(all_docs[i], all_docs[j])
    line.append(round(jaccard,2))
  matriz.append(line)
for i in range(1, len(matriz)):
  matriz[i].insert(0, matriz[0][i])
  
pandas.DataFrame(matriz)

,0,1,2,3,4
0,Measure,Binary,TF,TF-IDF,BM25
1,Binary,1,0,0,0
2,TF,0,1,0,0
3,TF-IDF,0,0,1,1
4,BM25,0,0,1,1
